# Instalando pacotes


In [ ]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 32 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minimal amd64 3.7.17-1+jammy1 [1,837 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd

In [ ]:
!sudo apt-get install python3.7-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3.7-lib2to3
The following NEW packages will be installed:
  python3.7-distutils python3.7-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 313 kB of archives.
After this operation, 1,229 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-lib2to3 all 3.7.17-1+jammy1 [124 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-distutils all 3.7.17-1+jammy1 [189 kB]
Fetched 313 kB in 0s (2,008 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize fron

In [ ]:
!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.5 MB/s eta 0:00:00


# Código

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS')

In [ ]:
%cd /content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS

/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS


In [ ]:
import glob
import logging
import os
import sys
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torchvision.datasets as dset

import utils as utils
from architect import Architect
from model_search import Network
from search_config import args



In [ ]:
def main():
    utils.set_seed(seed=0)
    print(f'gpu device = {args.gpu_id}')
    print(f'args = {args}')
    logging.info('gpu device = %d' % args.gpu_id)
    logging.info("args = %s", args)

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.cuda()
    model = Network(args, args.init_channels, CIFAR_CLASSES, args.layers, criterion)
    model = model.cuda()
    print(f'param size = {utils.count_parameters_in_MB(model)}')
    logging.info("param size = %fMB", utils.count_parameters_in_MB(model))

    optimizer = torch.optim.SGD(
        model.parameters(),
        args.learning_rate,
        momentum=args.momentum,
        weight_decay=args.weight_decay)
    architect = Architect(model, args)

    if args.dataset == 'cifar10':
        train_transform, valid_transform = utils.data_transforms_cifar10(args)
        train_data = dset.CIFAR10(root=args.data + args.dataset, train=True, download=True,
                                  transform=train_transform)
    elif args.dataset == 'cifar100':
        train_transform, valid_transform = utils.data_transforms_cifar100(args)
        train_data = dset.CIFAR100(root=args.data + args.dataset, train=True, download=True,
                                   transform=train_transform)
    elif args.dataset == 'brainMRI':
        ext_list = ['.jpg', '.JPG', '.jpeg', '.png']
        root_brain = '/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS/dataset/brain_tumor_dataset'

        train_transform, valid_transform = utils.data_transforms_brainMRI(args)
        train_data = dset.DatasetFolder(root = root_brain, loader =  utils.default_loader , extensions= ext_list, transform= train_transform)
    else:
        raise ValueError('No Defined Dataset!!!')

    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(args.train_portion * num_train))
    if args.debug:
        split = args.batch_size
        num_train = 2 * args.batch_size

    train_queue = torch.utils.data.DataLoader(
        train_data, batch_size=args.batch_size,
        sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
        pin_memory=True, num_workers=0)

    valid_queue = torch.utils.data.DataLoader(
        train_data, batch_size=args.batch_size,
        sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[split:num_train]),
        pin_memory=True, num_workers=0)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, float(args.epochs), eta_min=args.learning_rate_min)

    for epoch in range(args.epochs):
        scheduler.step()
        lr = scheduler.get_lr()[0]
        print(f'epoch {epoch} lr {lr}')
        logging.info('epoch %d lr %e', epoch, lr)

        # training
        train_acc, train_obj = train(train_queue, valid_queue, model, architect, criterion, optimizer, lr, epoch)
        print(f'train_acc {train_acc}')
        logging.info('train_acc %f', train_acc)

        # validation
        valid_acc, valid_obj = infer(valid_queue, model, criterion)
        print(f'valid_acc {valid_acc}')
        logging.info('valid_acc %f', valid_acc)

        # print arch params
        print(f'normal alpha = {F.softmax(model.alphas_normal, dim=-1)}')
        print(f'normal beta = {F.sigmoid(model.betas_normal)}')
        print(f'reduce alpha = {F.softmax(model.alphas_reduce, dim=-1)}')
        print(f'reduce beta = {F.sigmoid(model.betas_reduce)}')
        logging.info('normal alpha = %s', F.softmax(model.alphas_normal, dim=-1))
        logging.info('normal beta = %s', F.sigmoid(model.betas_normal))
        logging.info('reduce alpha = %s', F.softmax(model.alphas_reduce, dim=-1))
        logging.info('reduce beta = %s', F.sigmoid(model.betas_reduce))

        genotype = model.genotype()

        print(f'genotype = {genotype}')
        logging.info('genotype = %s', genotype)

        # utils.save(model, os.path.join(args.save, 'weights.pt'))
        utils.save_entire_model(model, os.path.join(args.save, 'weights.pt'))

def train(train_queue, valid_queue, model, architect, criterion, optimizer, lr, epoch):
    objs = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top5 = utils.AverageMeter()

    args.epoch = epoch
    if args.warmup and epoch < args.warmup_epoch:
        print(f'epoch {epoch} warming up!!!')
        logging.info('epoch %d warming up!!!', epoch)
    else:
        print(f'epoch {epoch} train arch!!!')
        logging.info('epoch %d train arch!!!', epoch)

    print('*' * 100)
    print('*' * 100)
    print(train_queue)
    print('*' * 100)
    print('*' * 100)

    for step, (input, target) in enumerate(train_queue):
        print(f'Step {step}/{len(train_queue)}')
        args.step = step

        model.train()
        n = input.size(0)
        input = input.cuda()
        target = target.cuda(non_blocking=True)

        print('\tStarting architecture search')

        if architect is not None:
            if args.warmup and epoch < args.warmup_epoch:
                pass
            else:
                # get a random minibatch from the search queue with replacement
                input_search, target_search = next(iter(valid_queue))
                input_search = input_search.cuda()
                target_search = target_search.cuda(non_blocking=True)

                architect.step(input, target, input_search, target_search, lr, optimizer, unrolled=args.unrolled)

        print('\tBackward do modelo')
        optimizer.zero_grad()
        logits = model(input)
        loss = criterion(logits, target)

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
        optimizer.step()

        print('\tAvaliando do modelo')
        prec1, prec5 = utils.accuracy(logits, target, topk=(1, 2))
        objs.update(loss.item(), n)
        top1.update(prec1.item(), n)
        top5.update(prec5.item(), n)

        # if (step == 10):
        #   break

        if step % args.report_freq == 0:
            print(f'train {step} {objs.avg} {top1.avg} {top5.avg}')
            logging.info('train %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)

    return top1.avg, objs.avg


def infer(valid_queue, model, criterion):
    objs = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top5 = utils.AverageMeter()
    model.eval()

    with torch.no_grad():
        for step, (input, target) in enumerate(valid_queue):
            input = input.cuda()
            target = target.cuda(non_blocking=True)

            logits = model(input)
            loss = criterion(logits, target)

            prec1, prec5 = utils.accuracy(logits, target, topk=(1, 2))
            n = input.size(0)
            objs.update(loss.item(), n)
            top1.update(prec1.item(), n)
            top5.update(prec5.item(), n)

            if step % args.report_freq == 0:
                print(f'valid {step} {objs.avg} {top1.avg} {top5.avg}')
                logging.info('valid %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)

    return top1.avg, objs.avg





In [ ]:
args.save = ''
if args.debug:
    args.save = 'log/debug-search-{}-{}'.format(args.save, time.strftime("%Y%m%d-%H%M%S"))
else:
    args.save = 'log/search-{}-{}-{}'.format(args.dataset, args.save, time.strftime("%Y%m%d-%H%M%S"))


if not os.path.exists('log'):
    os.mkdir('log')

utils.create_exp_dir(args.save, scripts_to_save=glob.glob('*.py'))
log_format = '%(asctime)s %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format=log_format, datefmt='%m/%d %I:%M:%S %p')
fh = logging.FileHandler(os.path.join(args.save, 'log.txt'))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

if args.dataset == 'cifar10':
    CIFAR_CLASSES = 10
elif args.dataset == 'cifar100':
    CIFAR_CLASSES = 100
elif args.dataset == 'brainMRI':
    CIFAR_CLASSES = 2
else:
    raise ValueError('No Defined Dataset!!!')


if __name__ == '__main__':
    utils.run_func(args, main)

Experiment dir : log/search-brainMRI--20240206-144420
gpu device = 0
args = Namespace(data='./dataset/', dataset='brainMRI', batch_size=8, lamda0=1.0, lamda1=1.0, learning_rate=0.025, learning_rate_min=0.001, momentum=0.9, weight_decay=0.0003, report_freq=50, gpu_id=0, epochs=50, init_channels=16, layers=8, model_path='saved_models', cutout=False, cutout_length=16, drop_path_prob=0.3, save='log/search-brainMRI--20240206-144420', seed=2, grad_clip=5, train_portion=0.5, debug=False, unrolled=False, del_none=True, aux_loss=True, beta_loss=True, info_linear_grow=False, one_level=False, arch_learning_rate=0.0003, arch_weight_decay=0.001, arch_name='du_darts_c10_s0', loss_type='entropy', warmup=False, warmup_epoch=25)
param size = 1.928562


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:809: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


A saída de streaming foi truncada nas últimas 5000 linhas.
Step 8/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 9/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 10/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 11/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 12/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 13/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 14/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
Step 15/16
	Starting architecture search
	Backward do modelo
	Avaliando do modelo
train_acc 80.15872991652716
valid 0 2.2016892433166504 37.5 100.0
valid_acc 32.28346459696612
normal alpha = tensor([[0.1395, 0.1393, 0.1400, 0.1462, 0.1459, 0.1444, 0.1447],
        [0.1423, 0.1426, 0.1435, 0.1426, 0.1437, 0.1428, 0.1425],
        [0.1405, 0.1398, 0.1405, 0.1453, 0.144